In [1]:
# Some dependencies 
using Pkg
Pkg.add("Symbolics")
Pkg.add("DataFrames")
using Symbolics
using DataFrames


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


## Fake position and bisection

In [2]:

function loop_and_search(f::Function, update_c::Function, a::Float64, b::Float64, tol::Float64, max_iter::Int64)
    
    
    """
    This function finds the root of a function f using the way of updating the midpoint c, 
    given by update_c and the interval [a,b]. The tolerance is given by tol. 
    The maximum number of iterations is given by max_iter.

    Parameters:
    ------------
        f: function which we want to find the root for
        update_c: function which updates the value of c (the midpoint, which should be very close to the root)
        a: left endpoint of the interval
        b: right endpoint of the interval
        tol: tolerance for the root
        max_iter: maximum number of iterations

        
    Returns:
    ------------
        c: midpoint of the interval, which should be very close to the root of f
        iters: number of iterations needed to find the root
    """


    c = update_c(f, a, b)
    value = f(c)
    iters = 1

    while (abs(value) > tol) && (iters < max_iter)

        if (value*f(a) < 0)
            b = c 
        elseif (value*f(b) < 0)
            a = c
        end

        c = update_c(f, a, b)
        value = f(c)
        iters += 1
    end

    return c, iters
end

loop_and_search (generic function with 1 method)

In [3]:

function update_c_bisection(f::Function, a::Float64, b::Float64)
    return (a+b)/2
end

update_c_bisection (generic function with 1 method)

In [4]:

function update_c_fake_position(f::Function, a::Float64, b::Float64)
    return (b - (f(b)*(b-a))/(f(b) - f(a)))
end

update_c_fake_position (generic function with 1 method)

In [5]:
function bisection_method_wrapper(f::Function,
                                  df::Function,
                                  x0::Float64,
                                  lower_bound::Float64,
                                  upper_bound::Float64,
                                  tol::Float64,
                                  max_iter::Integer)

    """
        This function is an standard wrapper for the bisection method. Some of the parameters won't be used, yet we 
        ask for them to be able to use it in the same way as the other methods. Whenever we later want to callit along 
        with the other methods.

        It calls the function loop_and_search with the corresponding parameters.
    """

    return loop_and_search(f, update_c_bisection, lower_bound, upper_bound, tol, max_iter)

end

bisection_method_wrapper (generic function with 1 method)

In [6]:
function fake_position_wrapper(f::Function,
                               df::Function,
                               x0::Float64,
                               lower_bound::Float64,
                               upper_bound::Float64,
                               tol::Float64,
                               max_iter::Integer)
    """
        This function is an standard wrapper for the fake position method. Some of the parameters won't be used, yet we 
        ask for them to be able to use it in the same way as the other methods. Whenever we later want to callit along 
        with the other methods.

        It calls the function loop_and_search with the corresponding parameters.
    """

    return loop_and_search(f, update_c_fake_position, lower_bound, upper_bound, tol, max_iter)
    
end

fake_position_wrapper (generic function with 1 method)

## Fixed point iteration

In [7]:
function check_constraints(f::Function, lower_bound::Float64, upper_bound::Float64)::Bool
   
    slope = abs( (f(upper_bound) - f(lower_bound))/(upper_bound-lower_bound) )

    if slope > 1
        return false
    else
        return true
    end
end

check_constraints (generic function with 1 method)

In [8]:
function fixed_point_iteration(f::Function,
                               x0::Float64,
                               lower_bound::Float64,
                               upper_bound::Float64,
                               tol::Float64,
                               maxiter::Int64)
    
    if  !check_constraints(f, lower_bound, upper_bound)
        return x0, maxiter
    end

    x = x0

    for i in 1:maxiter
        x = f(x)

        if (x - tol) < lower_bound || (x + tol) > upper_bound
            break

        elseif abs(f(x) - x) < tol
            println("The algorithm converged in $i iterations.")
            return x, i
        end
    end

    return x, maxiter
end

fixed_point_iteration (generic function with 1 method)

In [9]:
function fixed_point_iteration_wrapper(f::Function,
                                       df::Function,
                                       x0::Float64,
                                       lower_bound::Float64,
                                       upper_bound::Float64,
                                       tol::Float64,
                                       max_iter::Integer)

    """
        This function is an standard wrapper for the fixed point iteration method. Some of the parameters won't be used, yet we 
        ask for them to be able to use it in the same way as the other methods. Whenever we later want to callit along 
        with the other methods.

        It calls the function fixed_point_iteration with the corresponding parameters.
    """

    return fixed_point_iteration(f, x0, lower_bound, upper_bound, tol, max_iter)
end 

fixed_point_iteration_wrapper (generic function with 1 method)

## Newton Rapson

In [10]:
function newton_rapson_method(f::Function,
                              df::Function,
                              x0::Float64,
                              tol::Float64,
                              maxiter::Integer)
    x = x0
    for i in 1:maxiter
        x = x - f(x)/df(x)
        if abs(f(x)) < tol
            return x, i
        end
    end
    return x, i
end

newton_rapson_method (generic function with 1 method)

In [16]:
function newton_rapson_method_wrapper(f::Function,
                                      df::Function,
                                      x0::Float64,
                                      lower_bound::Float64,
                                      upper_bound::Float64,
                                      tol::Float64,
                                      max_iter::Integer)
    """
        This function is an standard wrapper for the newton rapson method. Some of the parameters won't be used, yet we 
        ask for them to be able to use it in the same way as the other methods. Whenever we later want to callit along 
        with the other methods.

        It calls the function newton_rapson_method with the corresponding parameters.
    """

    return newton_rapson_method(f, df, x0, tol, max_iter)
end                                       

newton_rapson_method_wrapper (generic function with 1 method)


## 1. (valor 2pts) Emplear los métodos de solución de ecuaciones de una variable para dar solución a las siguientes ecuaciones:

1. $e^x − 4 + x = 0$
2. $x − 0.2 sin(x) − 0.5 = 0$
3. $\displaystyle e^{\frac{x}{2}} − x^2 − 3x = 0$
4. $e^x cos(x) − x^2 + 3x = 0$
5. $0.5x^3 + x^2 − 2x − 5 = 0$
6. $e^x − 4x^2 − 8x = 0$

In [12]:
@variables x
f1sym = exp(x) - 4 + x
f2sym = x - 0.2*Symbolics.sin(x) - 0.5
f3sym = exp(x/2) - x^2 - 3*x
f4sym = exp(x)*Symbolics.cos(x) - x^2 + 3x
f5sym = 0.5*x^3 + x^2 - 2*x - 5
f6sym = exp(x) - 4*x^2 - 8*x

fn = [f1sym, f2sym, f3sym, f4sym, f5sym, f6sym]

6-element Vector{Num}:
         x + exp(x) - 4
        x - 0.5 - 0.2sin(x)
             exp((1//2)*x) - 3x - (x^2)
 3x + cos(x)*exp(x) - (x^2)
      x^2 + 0.5(x^3) - 5 - 2x
             exp(x) - 8x - 4(x^2)

In [13]:
derivatives = [Symbolics.derivative(f, x) for f in fn]

6-element Vector{Num}:
        1 + exp(x)
       1 - 0.2cos(x)
     (1//2)*exp((1//2)*x) - 3 - 2x
 3 + cos(x)*exp(x) - 2x - exp(x)*sin(x)
      2x + 1.5(x^2) - 2
            exp(x) - 8 - 8x

In [14]:
compiledf = [Symbolics.build_function(f, x, expression = false) for f in fn]
compileddf = [Symbolics.build_function(df, x, expression = false) for df in derivatives]

6-element Vector{RuntimeGeneratedFunctions.RuntimeGeneratedFunction{(:x,), Symbolics.var"#_RGF_ModTag", Symbolics.var"#_RGF_ModTag", id, Expr} where id}:
 RuntimeGeneratedFunctions.RuntimeGeneratedFunction{(:x,), Symbolics.var"#_RGF_ModTag", Symbolics.var"#_RGF_ModTag", (0xa9d56c03, 0xf763ce0d, 0x69b02d88, 0xcb0eacb9, 0xc42076d2), Expr}(quote
    #= /home/dave/.julia/packages/SymbolicUtils/Oyu8Z/src/code.jl:373 =#
    #= /home/dave/.julia/packages/SymbolicUtils/Oyu8Z/src/code.jl:374 =#
    #= /home/dave/.julia/packages/SymbolicUtils/Oyu8Z/src/code.jl:375 =#
    (+)(1, (exp)(x))
end)
 RuntimeGeneratedFunctions.RuntimeGeneratedFunction{(:x,), Symbolics.var"#_RGF_ModTag", Symbolics.var"#_RGF_ModTag", (0xa1abfe37, 0xd0223307, 0x7cd529de, 0xd4371e2a, 0x330b6ea7), Expr}(quote
    #= /home/dave/.julia/packages/SymbolicUtils/Oyu8Z/src/code.jl:373 =#
    #= /home/dave/.julia/packages/SymbolicUtils/Oyu8Z/src/code.jl:374 =#
    #= /home/dave/.julia/packages/SymbolicUtils/Oyu8Z/src/code.jl:375 =#


Ahora que tenemos nuestras funciones y sus derivadas compiladas podemos felizmente tratar de mirar hacia dónde convergen, y hacernos una idea sobre los métodos. Haremos dos tests, el primero es en "igualdad" de condiciones iniciales, es decir, todas van a empezar con los mismos:

- intervalos
- puntos iniciales
- tolerancia
- máximo número de iteraciones

Con esto buscamos evaluar qué tanto nos pueden servir las funciones en un primer intento asumiendo que no podemos saber mucho sobre la función y que por lo tanto no podemos escoger unos mejores puntos iniciales, o intervalos.

In [18]:
x = 0.1 # Initial guess
lower_bound = -10.0
upper_bound = 10.0
tol = 1e-10
max_iter = 1000

methods = [(bisection_method_wrapper, "Bisection"), (fake_position_wrapper, "Fake position"),
           (fixed_point_iteration_wrapper, "Fixed point iteration"), (newton_rapson_method_wrapper, "Newton rapson")]

#run all the methods for all the functions
results = []

for i in 1:length(fn)
    for method in methods
        push!(results, method[1](compiledf[i], compileddf[i], x, lower_bound, upper_bound, tol, max_iter))
    end
end

Tenga en cuenta que si en alguna función se llegó al máximo de iteraciones (1000) puede que haya tenido grandes problemas convergiendo, y si además el x al que llegó es el x0 (0.1) entonces esto se traduce en que no pasó de la primera iteración por que no cumplía condiciones.

In [23]:
performances = DataFrame(name = [methods[2] for i in 1:length(fn) for methods in methods],
                         x = [results[i][1] for i in 1:length(results)],
                         iterations = [results[i][2] for i in 1:length(results)])

#sort by name and then by iterations
sort!(performances, [:name, :x])

Row,name,x,iterations
,String,Float64,Int64
1,Bisection,0.615468,32
2,Bisection,2.11794,37
3,Bisection,1.07373,38
4,Bisection,-3.07017,38
5,Bisection,-0.236947,38
6,Bisection,-2.0165,38
7,Fake position,0.615468,14
8,Fake position,9.58565,16
9,Fake position,2.11794,260


### Análisis de resultados (Punto 1 - parte 1)

AAAAAAAAAAAAAAAAAAAAAA hay que escribir carreta

Ahora vamos a hacer intervalos razonables para cada función asumiendo que la conocemos